# FIT5202 Assignment 1 : Analysing eCommerce Data

## Table of Contents
* 
    * [Part 1 : Working with RDD](#part-1)
        * [1.1 Data Preparation and Loading](#1.1)
        * [1.2 Data Partitioning in RDD](#1.2)
        * [1.3 Query/Analysis](#1.3)
    * [Part 2 : Working with DataFrames](#2-dataframes)
        * [2.1 Data Preparation and Loading](#2-dataframes)
        * [2.2 Query/Analysis](#2.2)
    * [Part 3 :  RDDs vs DataFrame vs Spark SQL](#part-3)

# Part 1 : Working with RDDs <a class="anchor" name="part-1"></a>
## 1.1 Working with RDD
In this section, you will need to create RDDs from the given datasets, perform partitioning in these RDDs and use various RDD operations to answer the queries for retail analysis. 

### 1.1.1 Data Preparation and Loading <a class="anchor" name="1.1"></a>
Write the code to create a SparkContext object using SparkSession. To create a SparkSession you first need to build a SparkConf object that contains information about your application, use Melbourne time as the session timezone. Give an appropriate name for your application and run Spark locally with as many working processors as logical cores on your machine. 

In [1]:
# Import SparkConf class into program
from pyspark import SparkConf

# local[*]: run Spark in local mode with as many working processors as logical cores on your machine
# If we want Spark to run locally with 'k' worker threads, we can specify as "local[k]".
master = "local[*]"
# The `appName` field is a name to be shown on the Spark cluster UI page
app_name = "Assignment 1"
# Setup configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name)

# Import SparkContext and SparkSession classes
from pyspark import SparkContext # Spark
from pyspark.sql import SparkSession # Spark SQL

# Method 1: Using SparkSession
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')

1.1.2 Load CUPS csv files into four RDDs.

In [7]:
import csv

In [2]:
category_rdd = sc.textFile('./sales/category.csv')
users_rdd = sc.textFile('./users.csv')
product_rdd = sc.textFile('./sales/product.csv')
sales_rdd = sc.textFile('./sales/sales.csv')

1.1.3 For each RDD, remove the header rows and display the total count and first 10 records. (Hint: You can use csv.reader to parse rows into RDDs.)

In [13]:
category_header

'category_id,category_code'

In [3]:
category_header = category_rdd.first()
category_rdd = category_rdd.filter(lambda row: row != category_header)
category_rdd.take(10)

['2090971686529663114,appliances.environment.vacuum',
 '2232732116498514828,apparel.jeans',
 '2232732109628244704,apparel.shirt',
 '2232732103294845523,apparel.shoes.step_ins',
 '2232732086500851925,apparel.scarf',
 '2232732100660822557,country_yard.cultivator',
 '2053013558282682943,construction.tools.drill',
 '2053013562527318829,furniture.living_room.cabinet',
 '2110937143172923797,construction.tools.light',
 '2074462942123786261,kids.toys']

In [14]:
users_header

'user_id,firstname,lastname,dob,address,postcode,state,sex,suburb'

In [4]:
users_header = users_rdd.first()
users_rdd = users_rdd.filter(lambda row: row != users_header)
users_rdd.take(10)

['514771925,MERVYN,Unknown,1-5-1963,30A LUCINDA AVENUE,2260,NSW,male,WAMBERAL',
 '531947692,FALLON,Unknown,16-10-2002,82 CAMBERWARRA DRIVE,6025,WA,female,CRAIGIE',
 '560231306,KALIN,Unknown,19-8-2015,74 SOLDIERS ROAD,2281,NSW,male,PELICAN',
 '575248835,PIETTA,Unknown,19-1-1962,"UNIT 1, 1 LARK AVENUE",5023,SA,female,SEATON',
 '575243330,SUMMAH,Unknown,19-4-1973,44 CLARENDON ROAD,2048,NSW,female,STANMORE',
 '516606243,KEIRA,Unknown,9-6-2011,1 VIOLA CLOSE,4868,QLD,female,BAYVIEW HEIGHTS',
 '545807768,VERITY,Unknown,1-2-1980,31-33 KING STREET,3550,VIC,female,BENDIGO',
 '529118965,MAXWELL,Unknown,16-5-2012,15 HARWELL WAY,6721,WA,male,WEDGEFIELD',
 '518625399,NATASHIA,Unknown,5-6-1973,"UNIT 3, 17 BOX STREET",6530,WA,female,WEBBERTON',
 '518273873,JUSTINE,Unknown,27-8-2007,"UNIT 909, 112 GODERICH STREET",6004,WA,male,EAST PERTH']

In [15]:
product_header

'product_id,category_id,brand,avg_cost'

In [5]:
product_header = product_rdd.first()
product_rdd = product_rdd.filter(lambda row: row != product_header)
product_rdd.take(10)

['100168127,2053013554096767303,,30.8171196068388',
 '7101887,2232732103764607583,,33.6457534471799',
 '100058603,2053013555438944659,,32.1982006550096',
 '100007811,2232732112782361392,etro,300.111251624047',
 '26400508,2053013553056579841,,139.458912588293',
 '100215722,2134905019189691101,,4.15035271832127',
 '32402016,2232732115777094520,,5.90104832419485',
 '13201248,2232732061804790604,brw,400.468187201711',
 '2701879,2053013563911439225,beko,249.969115457257',
 '15300266,2232732107698864813,,2.85509519214549']

In [16]:
sales_header

'id,sales_timestamp,product_id,price,user_id,user_session'

In [6]:
sales_header = sales_rdd.first()
sales_rdd = sales_rdd.filter(lambda row: row != sales_header)
sales_rdd.take(10)

['338156802,2020-03-27 02:03:48.000000 +00:00,4804056,171.56,513119357,0ad58441-9db9-48bd-9ee7-accf603f06e0',
 '338156808,2020-03-27 02:03:49.000000 +00:00,1005223,209.53,522277649,2ecb2725-2c40-4acb-8ef1-05e39ea4c2cc',
 '338156913,2020-03-27 02:04:11.000000 +00:00,1005212,174.25,632847510,43609582-1818-4885-bd72-3791f3a4ec93',
 '338156921,2020-03-27 02:04:13.000000 +00:00,1005236,231.15,626579483,bbf972ef-8f8d-4a33-bcbd-ac09af67ea40',
 '338156953,2020-03-27 02:04:18.000000 +00:00,1005212,174.25,633245502,90166e0d-d7da-4997-b479-bc3a0fd724d6',
 '338156985,2020-03-27 02:04:24.000000 +00:00,100132156,47.5,541680528,4cd111cc-8716-4720-b7e3-bc3c8ddce98f',
 '338157055,2020-03-27 02:04:37.000000 +00:00,3500009,290.84,618162129,d0bda424-a759-4941-b9be-5091c28009db',
 '338157122,2020-03-27 02:04:48.000000 +00:00,2300214,386.08,626601722,ce913646-936a-4b2e-b6bb-9f6e0d781bc5',
 '338157141,2020-03-27 02:04:52.000000 +00:00,100070443,35.01,604369152,fd55e2cd-dc42-4bf9-a0a1-f207f1c66e5d',
 '3381572

1.1.4 Drop unnecessary columns from RDDs: firstname, lastname, user_session.

### 1.2 Data Partitioning in RDD <a class="anchor" name="1.2"></a>
1.2.1 For each RDD, print out the total number of partitions and the number of records in each partition. Answer the following questions: How many partitions do the above RDDs have? How is the data in these RDDs partitioned by default, when we do not explicitly specify any partitioning strategy? Can you explain why it will be partitioned in this number? If I only have one single core CPU in my PC, what is the default partition's number? (Hint: search the Spark source code to try to answer this question.) Write code and your explanation in Markdown cells.

1.2.2. Create a user defined function (UDF) to transform category_code to capitalized words. (e.g. apparel.shoes.ballet_shoes shall be converted to “Apparel Shots Ballet_shoes).

1.2.3. Join Product and Category RDDs and Create a new key value RDD, using brand as the key and all of the categories of that brand as the value. Print out the first 5 records of the key-value RDD.

### 1.3 Query/Analysis <a class="anchor" name="1.3"></a>
For this part, write relevant RDD operations to answer the following queries.

**1.3.1 Calculate the average daily sales for each year, each month. Print the results as the following format(see assignment specification).**

**1.3.2 Find 10 of the best selling brands. You should display the brand and total revenue in the result.**

## Part 2. Working with DataFrames <a class="anchor" name="2-dataframes"></a>
In this section, you will need to load the given datasets into PySpark DataFrames and use DataFrame functions to answer the queries.
### 2.1 Data Preparation and Loading

**2.1.1. Load CUPS into four separate dataframes. When you create your dataframes, please refer to the metadata file and think about the appropriate data type for each columns (Note: Initially, you should read date/time related column as the string type).**

**2.1.2 Display the schema of the four dataframes.**

### 2.2 QueryAnalysis  <a class="anchor" name="2.2"></a>
Implement the following queries using dataframes. You need to be able to perform operations like filtering, sorting, joining and group by using the functions provided by the DataFrame API.   

**2.2.1. Transform the 'sales_time' column in the sales dataframe to the date type; extract the hour in sales_date and create a new column “sales_hour”; after that, show the schema.**

**2.2.2. Calculate total sales for each hour, sort your result based on each hour’s sales in a descending order. Print out the sales_hour and total_sales columns.**

**2.2.3.Find 10 most profitable categories (profit can be simply defined as price - avg_cost). Print out the category name and total profit. Please print the category name in capitalized word format(hint: you can reuse the UDF defined in part 1.)**

**2.2.4. Use DataFrame filters to find all transactions sold at loss (defined as price < avg_cost), calculate 10 worst loss margin in percentage. (margin is defined as (price - avg_cost)/avg_cost; if price - avg_cost > 0, it’s call a profit margin; otherwise a loss margin)**

**2.2.5. Draw a barchart to show total sales from different states in each year.**

**2.2.6. Draw a scatter plot of customer age and their total spending with MOTH. To limit the number of datapoints, you may show the top 1000 “most valuable” customers only. You may also use log scale for the XY axis.**

### Part 3 RDDs vs DataFrame vs Spark SQL (15%) <a class="anchor" name="part-3"></a>
Implement the following queries using RDDs, DataFrames in SparkSQL separately. Log the  time taken for each query in each approach using the “%%time” built-in magic command in Jupyter Notebook and discuss the performance difference between these 3 approaches. 


#### Query: Find top 100 most popular products (by total sales) among user age group 20-40, group by brand, and show total sales revenue of each brand.

#### 3.1. RDD Implementation

#### 3.2. DataFrame Implementation

#### 3.3. Spark SQL Implementation

### 3.4 Observe the query execution time among RDD, DataFrame, SparkSQL, which is the fastest and why? (Maximum 500 words.)

#### Summary

The time might differ when running on different machines. Overall, the DataFrame and SQL queries run ...... compared with RDD.


### Some ideas on the comparison

Armbrust, M., Huai, Y., Liang, C., Xin, R., & Zaharia, M. (2015). Deep Dive into Spark SQL’s Catalyst Optimizer. Retrieved September 30, 2017, from https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html

Damji, J. (2016). A Tale of Three Apache Spark APIs: RDDs, DataFrames, and Datasets. Retrieved September 28, 2017, from https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

Data Flair (2017a). Apache Spark RDD vs DataFrame vs DataSet. Retrieved September 28, 2017, from http://data-flair.training/blogs/apache-spark-rdd-vs-dataframe-vs-dataset

Prakash, C. (2016). Apache Spark: RDD vs Dataframe vs Dataset. Retrieved September 28, 2017, from http://why-not-learn-something.blogspot.com.au/2016/07/apache-spark-rdd-vs-dataframe-vs-dataset.html

Xin, R., & Rosen, J. (2015). Project Tungsten: Bringing Apache Spark Closer to Bare Metal. Retrieved September 30, 2017, from https://databricks.com/blog/2015/04/28/project-tungsten-bringing-spark-closer-to-bare-metal.html